In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os 

#创建一个reader来读取TFRecord文件中的样例
reader = tf.TFRecordReader()
filename = tf.train.string_input_producer(['./data/tfrecord/test.tfrecords'])
# filename = tf.train.string_input_producer(['./data/tfrecord/validation.tfrecords'])

#从文件中读取一个样例
_,one_example = reader.read(filename)
#解析读入的数据
features = tf.parse_single_example(one_example,features={
    'labels':tf.FixedLenFeature([],tf.int64),
    'image_raw':tf.FixedLenFeature([],tf.string),
    'img_width':tf.FixedLenFeature([],tf.int64),
    'img_high':tf.FixedLenFeature([],tf.int64)
})
#将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'],tf.uint8)
images = tf.reshape(images, [48, 64, 1])
labels = tf.cast(features['labels'],tf.int32)
width = tf.cast(features['img_width'],tf.int32)
high = tf.cast(features['img_high'],tf.int32)

images_bath, labels_bath = tf.train.shuffle_batch([images, labels], batch_size = 128, capacity = 4000, 
                                                  min_after_dequeue = 3888, 
                                                  num_threads = 2,
                                                  shapes = ([48,64,1],[]))


In [2]:
learning_rate = 0.001
def hot(x):
    b =[]
    y = np.zeros([x.shape[0],4,11])
    for i in range(x.shape[0]):
        c= []
        b.append(str(x[i]))
        for j in range(len(b[i])):
            c.append(int(b[i][j]))
        while len(c)<4:
            c.append(10)
        for k in range(4):
            y[i][k][c[k]] = 1
    return y
X = tf.placeholder(tf.float32, [None, 48, 64, 1])
Y = tf.placeholder(tf.float32, [None, 4, 11])
# images=tf.image.convert_image_dtype(image,tf.float32) 
# X = images
# Y = hot(label,100)
with tf.variable_scope('layer1'):
    W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 16], stddev = 0.1)) 
    b1 = tf.Variable(tf.constant(0.0, shape=[16]))
    conv1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
    Y0 = tf.nn.relu(tf.nn.bias_add(conv1, b1))
    pool0 = tf.nn.max_pool(Y0,ksize=[1,2,2,1], strides=[1,2,2,1],padding = 'SAME')
    
    
with tf.variable_scope('layer2'):
    W2 = tf.Variable(tf.truncated_normal([5,5,16,32], stddev = 0.1))
    b2 = tf.Variable(tf.constant(0.0, shape=[32]))
    conv2 = tf.nn.conv2d(Y0, W2, strides = [1,1,1,1], padding = 'SAME') 
    Y1 = tf.nn.relu(tf.nn.bias_add(conv2, b2))
    pool1 = tf.nn.max_pool(Y1,ksize=[1,2,2,1], strides=[1,2,2,1],padding = 'SAME')
    
    
with tf.variable_scope('layer3'):
    W3 = tf.Variable(tf.truncated_normal([3, 5, 32, 48], stddev = 0.1))
    b3 = tf.Variable(tf.constant(0.0, shape = [48]))
    conv3 = tf.nn.conv2d(pool1, W3, strides = [1,1,1,1], padding = 'SAME')
    Y2 = tf.nn.relu(tf.nn.bias_add(conv3, b3))
    pool2 = tf.nn.max_pool(Y2,ksize=[1,2,2,1], strides=[1,2,2,1],padding = 'SAME')
    reshape = tf.reshape(pool2,[-1,12*16*48])
#     pool_size = tf.shape(pool2)
#     reshape = tf.reshape(pool2,[pool_size[0],-1])
with tf.variable_scope('layer4'):
    fc1_weight = tf.Variable(tf.truncated_normal([12*16*48,512],stddev = 0.1))
    fc1_bias=tf.Variable(tf.constant(0.0,shape=[512]))
    fc1 = tf.nn.relu(tf.matmul(reshape,fc1_weight)+fc1_bias)
    
    
with tf.variable_scope('layer5'):
    fc2_weight = tf.Variable(tf.truncated_normal([512,44], stddev = 0.1))
    fc2_bias=tf.Variable(tf.constant(0.0,shape=[44]))
    fc2 = tf.matmul(fc1,fc2_weight)+fc2_bias
    fc21 = tf.reshape(fc2,[-1,4,11])
    
    
with tf.variable_scope('cross_entropy'):
    cross_entropy1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y[:,0], logits=fc21[:,0]))
    cross_entropy2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y[:,1], logits=fc21[:,1]))
    cross_entropy3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y[:,2], logits=fc21[:,2]))
    cross_entropy4 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y[:,3], logits=fc21[:,3]))
    cross_entropy = (cross_entropy1+cross_entropy2+cross_entropy3+cross_entropy4) / 4.
# is_correct1 = tf.equal(tf.argmax(fc21[:,0]), tf.argmax(Y[:,0]))
# is_correct2 = tf.equal(tf.argmax(fc21[:,1]), tf.argmax(Y[:,1]))
# is_correct3 = tf.equal(tf.argmax(fc21[:,2]), tf.argmax(Y[:,2]))
# is_correct4 = tf.equal(tf.argmax(fc21[:,3]), tf.argmax(Y[:,3]))
is_correct = tf.equal(tf.argmax(fc21,2), tf.argmax(Y,2))
# accuracy1 = tf.reduce_mean(tf.cast(is_correct1, tf.float32))
# accuracy2 = tf.reduce_mean(tf.cast(is_correct2, tf.float32))
# accuracy3 = tf.reduce_mean(tf.cast(is_correct3, tf.float32))
# accuracy4 = tf.reduce_mean(tf.cast(is_correct4, tf.float32))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
# train_step1 = optimizer.minimize(cross_entropy1)
# train_step2 = optimizer.minimize(cross_entropy2)
# train_step3 = optimizer.minimize(cross_entropy3)
# train_step4 = optimizer.minimize(cross_entropy4)
train_step = optimizer.minimize(cross_entropy)
# saver
cnn_txt = "./cnn_txt"
if not os.path.exists(cnn_txt):
    os.makedirs(cnn_txt)
saver = tf.train.Saver()
with tf.Session() as sess:
    #启动多线程
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord = coord)
    txt = tf.train.get_checkpoint_state(cnn_txt)
    if txt and txt.model_checkpoint_path:
        print(txt.model_checkpoint_path)
        saver.restore(sess, txt.model_checkpoint_path)
    for epoch in range(100):
        # Load the input data
        image, label = sess.run([images_bath, labels_bath])
        train_data = {X: image / 255, Y: hot(label)}
        # Train
        sess.run([train_step],feed_dict = train_data)
#         sess.run([ train_step1, train_step2, train_step3, train_step4], feed_dict = train_data)
        # Accuracy on training data
#         acc1, acc2, acc3, acc4 = sess.run([accuracy1, accuracy2, accuracy3, accuracy4], feed_dict = train_data)
        acc, loss = sess.run([accuracy, cross_entropy],feed_dict = train_data)
#         saver.save(sess,cnn_txt + "/model.ckpt")
        if epoch % 10 ==0:
            print("epoch: %d ,accuracy is: %g loss is : %g" % (epoch, acc, loss))
#             print("acc1: %g, acc2: %g, acc3: %g, acc4: %g" % (acc1, acc2, acc3, acc4))
#             print("loss1: %g, loss2: %g, loss3: %g loss4: %g" % (loss1,loss2,loss3,loss4))
#     print("acc_finally: %g" % acc_fin)
    coord.request_stop()
    coord.join(threads)

./cnn_txt/model.ckpt
INFO:tensorflow:Restoring parameters from ./cnn_txt/model.ckpt
epoch: 0 ,accuracy is: 0.753906 loss is : 0.85972
epoch: 10 ,accuracy is: 0.751953 loss is : 0.739567
epoch: 20 ,accuracy is: 0.818359 loss is : 0.599244
epoch: 30 ,accuracy is: 0.792969 loss is : 0.60903
epoch: 40 ,accuracy is: 0.822266 loss is : 0.559406
epoch: 50 ,accuracy is: 0.837891 loss is : 0.473065
epoch: 60 ,accuracy is: 0.888672 loss is : 0.351295
epoch: 70 ,accuracy is: 0.904297 loss is : 0.3262
epoch: 80 ,accuracy is: 0.921875 loss is : 0.230892
epoch: 90 ,accuracy is: 0.943359 loss is : 0.207615
